<a href="https://colab.research.google.com/github/archei2500/overdubbing_courses/blob/main/lip_sync.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> **Добро пожаловать в интерактивную систему для синхронизации движений губ с речью!**

> Здесь вы можете выполнить синхронизацию движений губ с речью в видео на основе файла с субтитрами (расшифровкой) с временными метками.

> Пожалуйста, следуйте инструкциям.
1.   Для начала перейдите к первому разделу, укажите, нужно ли вам автоматическое улучшение качества после обработки (после анимации губ качество получается хуже), и запустите ячейку. В среду установится всё необходимое для работы.
2.   Перейдите ко второму разделу: загрузите видео и текст с субтитрами (текстом видео с временными метками) в систему любым удобным способом из предложенных.
3.   В разделе ниже выполните синхронизацию губ с речью, выбрав номера фраз из вашего файла с субтитрами, модель и некоторые параметры по желанию. Ожидайте результата.

>О моделях для синхронизации губ:
1.  Классический Wav2Lip. В данном случае движение губ хорошо синхронизируется с аудиодорожкой. Недостаток - качество изображения на месте губ получается очень плохим. Но после этого можно воспользоваться нейросетью для улучшения качества изображения.
2. Wav2Lip + GAN. Движение губ синхронизируется с аудиодорожкой хуже. Небольшая амплитуда открывания рта. Качество изображения лучше по сравнению с Wav2Lip без GAN. Выглядит наиболее естественно.
3. VideoReTalking. Амплитуда открывания рта больше. Качество изображения низкое. Также желательно применить далее нейросеть для улучшения качества изображения.

> *Примечание: в каждом кадре указанных фрагментов должно быть лицо.*

> ***Убедитесь в том, что вы подключены к среде с GPU!!!***

# Запустить перед началом работы

In [ ]:
# @title Нужно ли улучшать качество видео после обработки?
gfpgan = False # @param {type: "boolean"}

!pip install ffmpeg-python
!pip install moviepy
!pip install pydub
!pip install torchvision==0.16.2 # более старая версия для совместимости
if gfpgan:
  # установка GFPGAN для улучшения качества кадров
  !git clone https://github.com/TencentARC/GFPGAN.git

  !pip install basicsr
  !pip install facexlib
  !pip install -r requirements.txt
  !python setup.py develop
  !pip install realesrgan # используется для увеличения разрешения фоновых (не лицевых) областей

  # Загрузка предобученной модели
  !wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models

  import numpy as np
  import glob
  import subprocess
from pydub import AudioSegment
from IPython.display import clear_output
import os
from google.colab import files
import ffmpeg
import re
from moviepy.editor import VideoFileClip, AudioFileClip, concatenate_videoclips
import cv2
from tensorflow.test import gpu_device_name


def str_to_time(s):
  return int(s[6:8]) + 60 * int(s[3:5]) + 3600 * int(s[:2]) + float('0.' + s[9:12])


def time_to_str(time):
  hours = int(time // 3600)
  minutes = int((time - 3600 * hours) // 60)
  seconds = time - 60 * minutes
  str_sec = str(round(seconds, 3)).replace('.', ',')
  return str(hours).zfill(2) + ':' + str(minutes).zfill(2) + ':' + str_sec[:str_sec.find(',')].zfill(2) + ',' + str_sec[str_sec.find(',') + 1:].ljust(3, '0')


def extract_number(s):
  match = re.search(r'\d+', s) # регулярное выражение \d+ означает "одна или более цифр подряд"
  if match:
    return int(match.group())
  else:
    return 0


def convert_frames_to_video(path_in, path_out, fps):
  frame_array = []
  files = [join(path_in, f) for f in os.listdir(path_in) if isfile(join(path_in, f))]
  files = sorted(files, key = extract_number)

  for file in files:
    # чтение файлов с кадрами
    img = cv2.imread(file)
    height, width, layers = img.shape
    size = (width, height)
    # добавление кадров в список изображений
    frame_array.append(img)
  out = cv2.VideoWriter(path_out, cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
  for i in range(len(frame_array)):
    # запись списка изображений в качестве видео
    out.write(frame_array[i])
  out.release()


wav2lip_installed = False
vidretalk_installed = False

path_to_text = '/content/subtitles.srt'
path_to_video = '/content/vid.mp4'
path_to_auds = '/content/aud_fragms'
path_to_vids = '/content/vid_fragms'
path_to_lip_res = '/content/lip_results'

# проверка того, что пользователь подключен к среде GPU
device_name = gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU не найден! Совет: подключитесь к среде с GPU.')

clear_output()

# Загрузка видео и файла с субтитрами (расшифровкой)

In [ ]:
# @title Загрузка видео
#@markdown ### Способ загрузки
upload_method = "С устройства" #@param ["С устройства", "По ссылке с YouTube", "Путь к файлу Google Drive"]
#@markdown <font color="orange"> Примечания и советы:

#@markdown <font color="orange"> ``1. При выборе загрузки с устройства нужно будет нажать на кнопку в выходных данных и выбрать файл с устройства.``

#@markdown <font color="orange"> ``2. Для загрузки с YouTube укажите ссылку в соответствующее поле.``

#@markdown <font color="orange"> ``3. Обратите внимание: при выборе способа, связанного с Google Диском, система попросит вас дать разрешение на доступ к вашему Google Drive. Затем он будет смонтирован в файловую систему. После этого укажите путь к файлу в соответствующем поле.``

#@markdown ``Вставьте ссылку на видеоролик с платформы YouTube в требуемом формате `` 👇
youtube_url = 'https://www.youtube.com/watch?v=YOUTUBE_ID' #@param {type:"string"}

#@markdown ``Введите полный путь к видео на вашем Google Диске (для варианта с загрузкой с Google Drive) `` 👇
path_google_drive = '/content/drive/MyDrive/path_to_vid' #@param {type:"string"}

from urllib import parse as urlparse

# Удаление файлов, если такие уже были в ФС прежде
if os.path.isfile(path_to_video):
    os.remove(path_to_video)
if os.path.isfile('/content/aud.wav'):
    os.remove('/content/aud.wav')

path_to_video = '/content/vid.mp4' # путь к видео по умолчанию

# Видеофайл загружается разными способами
if upload_method == "С устройства":
  uploaded = files.upload()
  for filename in uploaded.keys():
    if '/content/' + filename != path_to_video:
      os.rename(filename, path_to_video)  # переименование загруженного файла
elif upload_method == "По ссылке с YouTube":
  if not yt_downloaded:
    !pip install yt-dlp # установка при первом запуске
    yt_downloaded = True
  url_data = urlparse.urlparse(youtube_url)
  query = urlparse.parse_qs(url_data.query)
  YOUTUBE_ID = query["v"][0]
  # Загрузка видео с YouTube
  !yt-dlp -f "bestvideo[ext=mp4]+bestaudio[ext=wav]/best[ext=mp4]/best" --output "/content/vid.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID
  # Запоминаем путь к файлу
  for file in os.listdir('/content'):
    if file.startswith('vid'):
        path_to_video = os.path.join(directory, file)
else:
  if not gd_mounted:
    from google.colab import drive
    drive.mount("/content/drive") # монтирование Google Диска при первом запуске
    gd_mounted = True
  if not os.path.isfile(path_google_drive): # если файл не найден
      print("ERROR: File not found!")
      raise SystemExit(0)
  !cp $path_google_drive $path_to_video # копирование файла с Google Диска

# извлечение аудиодорожки из видео
video = VideoFileClip(path_to_video)
video.audio.write_audiofile('/content/aud.wav') # извлечение аудиодорожки

clear_output() # очистка выходных данных

In [ ]:
# @title Загрузка субтитров (файла со структурой SRT)

# @markdown Выберите метод загрузки и укажите путь к файлу, если привязываете Google Диск:
# @markdown ### Способ загрузки
upload_method = "С устройства" #@param ["С устройства", "Путь к файлу Google Drive"]
#@markdown <font color="orange"> Примечания и советы:

#@markdown <font color="orange"> ``1. При выборе загрузки с устройства нужно будет нажать на кнопку в выходных данных и выбрать файл с устройства.``

#@markdown <font color="orange"> ``2. Обратите внимание: при выборе способа, связанного с Google Диском, система попросит вас дать разрешение на доступ к вашему Google Drive. Затем он будет смонтирован в файловую систему. После этого укажите путь к файлу в соответствующем поле.``

#@markdown ``Введите полный путь к видео на вашем Google Диске (для варианта с загрузкой с Google Drive) `` 👇
path_google_drive = '/content/drive/MyDrive/path_to_subs' #@param {type:"string"}

# Удаление файла, если такой уже был в ФС прежде
if os.path.isfile(path_to_text):
    os.remove(path_to_text)

path_to_text = '/content/subtitles.srt' # путь к файлу по умолчанию

# Видеофайл загружается разными способами
if upload_method == "С устройства":
  uploaded = files.upload()
  for filename in uploaded.keys():
    if not (filename.endswith('.srt') or filename.endswith('.txt')):
      print("Файл должен иметь расширение SRT или TXT!")
      raise SystemExit(0)
    if '/content/' + filename != path_to_text:
      os.rename(filename, path_to_text) # переименование загруженного файла
else:
  if not gd_mounted:
    from google.colab import drive
    drive.mount("/content/drive") # монтирование Google Диска при первом запуске
    gd_mounted = True
  if not os.path.isfile(path_google_drive): # если файл не найден
    print("ERROR: File not found!")
    raise SystemExit(0)
  if not (path_google_drive.endswith('.srt') or path_google_drive.endswith('.srt')):
    print("Файл должен иметь расширение SRT или TXT!")
    raise SystemExit(0)
  !cp $path_google_drive $path_to_text # копирование файла с Google Диска

# Проверка на то, что файл имеет необходимую структуру
if check_txtfile(path_to_text):
  clear_output()
else:
  print('Неверная структура выбранного файла!')

# Синхронизация

In [ ]:
# @title Синхронизируйте движения губ с речью в выбранных фрагментах
# @markdown Укажите номера фрагментов из вашего файла с субтитрами (отсчёт от 1) через пробел:
timings_str = '1 32' # @param {type: "string"}
if timings_str:
  try:
    timings = [int(n) for n in timings_str.split()]
    timings.sort() # сортировка по возрастанию
  except ValueError:
    print('Вы некорректно ввели номера фрагментов. Попробуйте снова.')
    raise SystemExit(0)
else:
  print('Необходимо ввести номера фрагментов.')

# @markdown Выберите модель для анимации губ:
lip_tool = 'Wav2Lip' #@param ["Wav2Lip", "Wav2Lip + GAN", "VideoReTalking"]
# @markdown <font color = "orange"> Примечание: ``Учтите, что установка VideoReTalking может занимать около 15 минут.``

# @markdown ___

# @markdown ***Следующие настройки только для Wav2Lip или Wav2Lip + GAN:***

# @markdown Настройка рамки вокруг рта. Так регулируются отступы. Вы можете задействовать область подбородка, например, задав pad_bottom = 20.
pad_top = 0 #@param {type: "integer"}
pad_bottom = 0 #@param {type: "integer"}
pad_left = 0 #@param {type: "integer"}
pad_right = 0 #@param {type: "integer"}
# @markdown Чтобы не было чрезмерного сглаживания изображений лица, можно задать nosmooth:
nosmooth = True #@param {type: "boolean"}
# @markdown Можно уменьшить разрешение видео для лучшего результата, так как модели обучались на лицах с более низким разрешением. На это число будут нацело делиться ширина и высота каждого кадра.
resize = 1 #@param {type: "integer"}

# @markdown ___
# Скачать результат?
download_file = True #@param {type: "boolean"}

# проверка на то, что необходимые файлы были загружены
if os.path.isfile(path_to_video) and os.path.isfile(path_to_text):
  # чтение файла, чтобы взять оттуда тайминги по номерам
  txt_file = open(path_to_subs, 'r', encoding = 'utf-8')
  lines = [''] + txt_file.read().split('\n')
  txt_file.close()

  # проверка на принадлежность существующему диапазону таймингов, введённых пользователем
  if timings[0] >= 1 and timings[-1] <= len(lines) / 4:
    # создание папок для хранения аудио- и видеофрагментов
    !mkdir $path_to_auds
    !mkdir $path_to_vids

    # из видео вырезаются фрагменты для анимации губ
    for i, timing in enumerate(timings):
      start = str_to_time(lines[(timing - 1) * 4 + 2][:lines[(timing - 1) * 4 + 2].find(' ')])
      end = str_to_time(lines[(timing - 1) * 4 + 2][lines[(timing - 1) * 4 + 2].rfind(' ') + 1:])
      out_path = path_to_vids + '/' + str(i) + '.mp4'
      # фрагмент вырезается по таймингу
      !ffmpeg -y -i $video_path -ss {start + 0.001} -to {end} -async 1 $out_path
      # извлечение аудиодорожки с проверкой на то, что аудио и видео точно одной продолжительности
      video = VideoFileClip(out_path)
      video.audio.write_audiofile(path_to_auds + '/' + str(i) + '.wav') # извлечение аудиодорожки
      difference = video.duration - len(AudioSegment.from_file(path_to_auds + '/' + str(i) + '.wav')) / 1000
      if difference > 0:
        !rm $out_path
        !ffmpeg -y -i $video_path -to {video.duration - difference} -async 1 $out_path

    # фрагменты заносятся в списки и сортируются по возрастанию
    lip_fragments_list = [path_to_vids + '/' + item for item in os.listdir(path_to_vids) if '.ipynb' not in item]
    lip_fragments_list = sorted(lip_fragments_list, key = extract_number)
    aud_fragments_list = [path_to_auds + '/' + item for item in os.listdir(path_to_auds) if '.ipynb' not in item]
    aud_fragments_list = sorted(aud_fragments_list, key = extract_number)

    !mkdir $path_to_lip_res # создание папки для результатов lip sync

    if lip_tool == 'Wav2Lip' or lip_tool == 'Wav2Lip + GAN':
      if not wav2lip_installed:
        # установка wav2lip
        !git clone https://github.com/justinjohn0306/Wav2Lip
        %cd /content/Wav2Lip
        # загрузка предобученных моделей
        !wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/wav2lip.pth' -O 'checkpoints/wav2lip.pth'
        !wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/wav2lip_gan.pth' -O 'checkpoints/wav2lip_gan.pth'
        !wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/resnet50.pth' -O 'checkpoints/resnet50.pth'
        !wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/mobilenet.pth' -O 'checkpoints/mobilenet.pth'
        a = !pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl
        !pip install git+https://github.com/elliottzheng/batch-face.git@master
        %cd /content

        wav2lip_installed = True

      # выбирается одна из моделей
      checkpoint_path = 'checkpoints/wav2lip.pth' if lip_tool == 'Wav2Lip' else 'checkpoints/wav2lip_gan.pth'

      # синхронизация губ
      for i, lip_fragm in enumerate(lip_fragments_list):
        output_file_path = path_to_lip_res + '/' + str(i) + '.mp4'
        face_path = lip_fragm
        audio_path = aud_fragments_list[i]

        if nosmooth == False:
          command = f"cd Wav2Lip && python inference.py --checkpoint_path {checkpoint_path} --face {face_path} --audio {audio_path} --outfile {output_file_path} --pads {pad_top} {pad_bottom} {pad_left} {pad_right} --resize_factor {resize}"
          !{command}
        else:
          command = f"cd Wav2Lip && python inference.py --checkpoint_path {checkpoint_path} --face {face_path} --audio {audio_path} --outfile {output_file_path} --pads {pad_top} {pad_bottom} {pad_left} {pad_right} --resize_factor {resize} --nosmooth"
          !{command}

        # Увеличение размеров кадров после обработки, т. к. они уменьшаются
        vcap1 = cv2.VideoCapture(face_path) # старое видео до Wav2Lip
        vcap2 = cv2.VideoCapture(output_file_path) # новое видео после Wav2Lip
        fps = vcap2.get(cv2.CAP_PROP_FPS)
        new_width  = vcap1.get(cv2.CAP_PROP_FRAME_WIDTH) # float
        new_height = vcap1.get(cv2.CAP_PROP_FRAME_HEIGHT)
        out = cv2.VideoWriter('resized_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (int(new_width), int(new_height))) # создание объекта для записи видео

        while True:
          ret, frame = vcap2.read()
          if not ret:
              break

          # Изменение размеров кадра
          resized_frame = cv2.resize(frame, (int(new_width), int(new_height)))
          # Запись измененного кадра в новое видео
          out.write(resized_frame)
          #cv2.imwrite(resized_frame)

        # Освобождение ресурсов
        vcap1.release()
        vcap2.release()
        out.release()
        cv2.destroyAllWindows()

        os.remove(output_file_path)
        os.rename('resized_video.mp4', output_file_path)

    else:
      if not vidretalk_installed:
        !python --version
        !apt-get update
        !apt install ffmpeg &> /dev/null
        !git clone https://github.com/vinthony/video-retalking.git &> /dev/null
        %cd video-retalking
        !pip install -r requirements.txt
        !pip install numpy==1.23.0
        # проверить, какая зависимость нужна, т. к. что-то, связанное с torchvision, устарело
        #!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
        # загрузка предобученных моделей
        !mkdir ./checkpoints
        !wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/30_net_gen.pth -O ./checkpoints/30_net_gen.pth
        !wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/BFM.zip -O ./checkpoints/BFM.zip
        !wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/DNet.pt -O ./checkpoints/DNet.pt
        !wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/ENet.pth -O ./checkpoints/ENet.pth
        !wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/expression.mat -O ./checkpoints/expression.mat
        !wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/face3d_pretrain_epoch_20.pth -O ./checkpoints/face3d_pretrain_epoch_20.pth
        !wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/GFPGANv1.3.pth -O ./checkpoints/GFPGANv1.3.pth
        !wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/GPEN-BFR-512.pth -O ./checkpoints/GPEN-BFR-512.pth
        !wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/LNet.pth -O ./checkpoints/LNet.pth
        !wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/ParseNet-latest.pth -O ./checkpoints/ParseNet-latest.pth
        !wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/RetinaFace-R50.pth -O ./checkpoints/RetinaFace-R50.pth
        !wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/shape_predictor_68_face_landmarks.dat -O ./checkpoints/shape_predictor_68_face_landmarks.dat
        !unzip -d ./checkpoints/BFM ./checkpoints/BFM.zip
        %cd /content

        vidretalk_installed = True

        # синхронизация губ
        !cd video-retalking && python3 inference.py \
          --face {face_path} \
          --audio {audio_path} \
          --outfile {output_file_path}

    %cd /content
    # Улучшение качества кадров в видео с анимацией губ
    if gfpgan:
      !mkdir /content/results
      !mkdir /content/results_videos
      !mkdir /content/results_mp4_videos

      for i, timing in enumerate(timings):
        lip_fragm = '/content/lip_results/' + str(i) + '.mp4'

        if os.path.exists(lip_fragm):
          # преобразование видео в фреймы
          vid_stream = cv2.VideoCapture(lip_fragm)
          fps = vid_stream.get(cv2.CAP_PROP_FPS) # получаем fps

          !mkdir /content/vid_frames

          # сохранение всех кадров из видео
          current_frame = 0
          while (True):
            # cap.read() в ret возвращает значение типа boolean. Если frame прочитан корректно: ret = True.
            ret, frame = vid_stream.read()
            print(ret)

            if ret:
              frame_path = '/content/vid_frames/frame' + str(current_frame) + '.jpg'
              cv2.imwrite(frame_path, frame)
              # увеличение значения счётчика кадров
              current_frame += 1
            else:
              break

          vid_stream.release()
          cv2.destroyAllWindows()

          # улучшение кадров
          !cd GFPGAN && python inference_gfpgan.py -i /content/vid_frames -o /content/results -v 1.3 -s 1 --bg_upsampler realesrgan # на 1.3 поменять возможно потом, но для этого установить тоже 1.3 версию

          for f in os.listdir('/content/vid_frames'):
            os.remove(os.path.join('/content/vid_frames', f))

          # конвертирование super res frames to .avi
          path_out = "/content/results_videos/" + str(i) + '.avi'
          # fps видео
          convert_frames_to_video('/content/results/restored_imgs/', path_out, fps)

          for f in os.listdir('/content/results/restored_imgs'):
            os.remove(os.path.join('/content/results/restored_imgs', f))

          # конвертирование .avi to .mp4
          src = '/content/results_videos/'
          dst = '/content/results_mp4_videos/'

          for root, dirs, filenames in os.walk(src, topdown = False):
            for filename in filenames:
              try:
                _format = ''
                if ".flv" in filename.lower():
                    _format=".flv"
                if ".mp4" in filename.lower():
                    _format=".mp4"
                if ".avi" in filename.lower():
                    _format=".avi"
                if ".mov" in filename.lower():
                    _format=".mov"

                inputfile = os.path.join(root, filename)
                outputfile = os.path.join(dst, filename.lower().replace(_format, ".mp4"))
                subprocess.call(['ffmpeg', '-i', inputfile, outputfile])
              except:
                print("An exception occurred")

          for f in os.listdir('/content/results_videos'):
            os.remove(os.path.join('/content/results_videos', f))

      # Копирование улучшенных видео в lip_results
      for i, timing in enumerate(timings):
        lip_fragm = '/content/lip_results/' + str(i) + '.mp4'
        new_lip_fragm = '/content/results_mp4_videos/' + str(i) + '.mp4'
        !cp {new_lip_fragm} {lip_fragm}

    # Итоговая нарезка
    counter = 0
    for i, timing in enumerate(timings):
      lip_fragm = '/content/lip_results/' + str(i) + '.mp4'
      old_aud = '/content/aud_fragms/' + str(i) + '.wav'
      end = str_to_time(lines[(timing - 1) * 4 + 2][:lines[(timing - 1) * 4 + 2].find(' ')])
      out_path = '/content/fragments/' + str(counter) + '.mp4'
      aud_path = '/content/audios/' + str(counter) + '.wav'
      if len(timings) == 1: # единственный
        !ffmpeg -y -i $video_path -to {end} -async 1 $out_path
        audio = VideoFileClip(out_path).audio
        audio.write_audiofile(aud_path)
        counter += 1
        os.rename(lip_fragm, '/content/fragments/' + str(counter) + '.mp4')
        !cp {old_aud} {'/content/audios/' + str(counter) + '.wav'}
        counter += 1
        start = str_to_time(lines[(timing - 1) * 4 + 2][lines[(timing - 1) * 4 + 2].rfind(' ') + 1:])
        if VideoFileClip(video_path).duration > start:
          out_path = '/content/fragments/' + str(counter) + '.mp4'
          aud_path = '/content/audios/' + str(counter) + '.wav'
          !ffmpeg -y -i $video_path -ss {start + 0.001} -async 1 $out_path
          audio = VideoFileClip(out_path).audio
          audio.write_audiofile(aud_path)
      elif i == 0: # если самый первый
        !ffmpeg -y -i $video_path -to {end} -async 1 $out_path
        audio = VideoFileClip(out_path).audio
        audio.write_audiofile(aud_path)
        counter += 1
        os.rename(lip_fragm, '/content/fragments/' + str(counter) + '.mp4')
        !cp {old_aud} {'/content/audios/' + str(counter) + '.wav'}
        counter += 1
        start = str_to_time(lines[(timing - 1) * 4 + 2][lines[(timing - 1) * 4 + 2].rfind(' ') + 1:])
      elif i == len(timings) - 1: # если последний
        !ffmpeg -y -i $video_path -ss {start + 0.001} -to {end} -async 1 $out_path
        audio = VideoFileClip(out_path).audio
        audio.write_audiofile(aud_path)
        counter += 1
        os.rename(lip_fragm, '/content/fragments/' + str(counter) + '.mp4')
        !cp {old_aud} {'/content/audios/' + str(counter) + '.wav'}
        counter += 1
        start = str_to_time(lines[(timing - 1) * 4 + 2][lines[(timing - 1) * 4 + 2].rfind(' ') + 1:])
        if VideoFileClip(video_path).duration > start:
          out_path = '/content/fragments/' + str(counter) + '.mp4'
          aud_path = '/content/audios/' + str(counter) + '.wav'
          !ffmpeg -y -i $video_path -ss {start + 0.001} -async 1 $out_path
          audio = VideoFileClip(out_path).audio
          audio.write_audiofile(aud_path)
      else: # обычный
        end = str_to_time(lines[(timing - 1) * 4 + 2][:lines[(timing - 1) * 4 + 2].find(' ')])
        !ffmpeg -y -i $video_path -ss {start + 0.001} -to {end} -async 1 $out_path
        audio = VideoFileClip(out_path).audio
        audio.write_audiofile(aud_path)
        counter += 1
        os.rename(lip_fragm, '/content/fragments/' + str(counter) + '.mp4')
        !cp {old_aud} {'/content/audios/' + str(counter) + '.wav'}
        counter += 1
        start = str_to_time(lines[(timing - 1) * 4 + 2][lines[(timing - 1) * 4 + 2].rfind(' ') + 1:])

    fragments = ['/content/fragments/' + item for item in os.listdir('/content/fragments') if '.ipynb' not in item]
    fragments = sorted(fragments, key = extract_number)

    loaded_video_list = []
    # склеиваем все фрагменты
    for fragment in fragments:
      loaded_video_list.append(VideoFileClip(fragment))
    concatenate_clip = concatenate_videoclips(loaded_video_list, method = 'compose')
    concatenate_clip.write_videofile('/content/result.mp4', logger = None)

    # Накладываем аудиодорожку
    !ffmpeg -i /content/result.mp4 -i /content/aud.wav -c:v copy -c:a aac -strict experimental -map 0:v:0 -map 1:a:0 /content/final_video.mp4
    if download_file:
      files.download('/content/final_video.mp4')
    print('Готово. Результат вы найдёте по пути /content/final_video.mp4')
  else:
    print('Вы загрузили файл, который содержит ', int(len(times) / 4), ' фрагментов, но ввели ', timings_str)
else:
  if not os.path.isfile(path_to_video):
    print('Вы не загрузили видео. Пожалуйста, вернитесь к предыдущим ячейкам и загрузите его.')
  if not os.path.isfile(path_to_text):
    print('Вы не загрузили субтитры. Пожалуйста, вернитесь к предыдущим ячейкам и загрузите их.')

# Размонтировать Google Drive

In [ ]:
drive.flush_and_unmount()